## Chapter 18. asyncio를 이용한 동시성
Concurrency is Not Paralleism!. 일반적인 운용 환경에서도 100개 이상의 프로세스를 일상적으로 실행한다. 진짜 병렬로 처리하려면 CPU 코어가 여러 개 있어야 한다. 따라서 실제로는 대부분의 처리가 동시에 수행되지만 병렬로 수행되지는 않는다. 이 장에서는 이벤트 루프에 의해 운용되는 코루틴을 이용해서 동시성을 구현하는 asyncio 패키지에 대해 설명한다.

### 18.1 스레드와 코루틴 비교
파이선에는 스레드를 종료시키는 API가 정의되지 않지 않기 때문에 여기서는 singal.go 속성을 사용했다. 

In [8]:
""" [예제 18-1] spinner_thread.py : 스레드로 텍스트 스피너 애니메이트하기 """
import threading
import itertools
import time
import sys

class Signal:
    go = True
    
def spin(msg, signal):
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status)) # 문자열 길이만큼 백스페이스를 반복해서 커서를 앞으로 이동시킴
        time.sleep(.1)
        if not signal.go:
            break
    write(' ' * len(status) + '\x08' * len(status)) # 스페이스로 기존 문자열을 지우고 백스페이스로 원래 위치로 돌아감
    
def slow_function():
    # 입출력을 위해 장시간 기다리는 것 처럼 보임
    time.sleep(3) # 주 스레드에서 sleep( ) 함수를 호출할 때 GIL이 해제되므로 두 번째 스레드가 진행됨
    return 42

def supervisor():
    signal = Signal()
    spinner = threading.Thread(target=spin,
                               args=('thinking!', signal))
    print('spinner object:', spinner)
    spinner.start() # spinner 스레드 시작
    result = slow_function()
    signal.go = False
    spinner.join()  # spinner 스레드가 끝날 때 까지 기다림
    return result

def main():
    result = supervisor()
    print('Answer:', result)
   
if __name__ == '__main__':
    t0 = time.time()
    main()
    delta = time.time() - t0
    print(delta)

spinner object: <Thread(Thread-5, initial)>
| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking          Answer: 42
3.0417046546936035


스레드 대신 @asyncio.coroutine을 이용해서 동일한 동작을 어떻게 구현할 수 있는지 알아보자
```
asyncio는 '코루틴'을 더욱 엄격히 정의한다. asyncio API에 사용할 코루틴은 본체 안에서 yield가 아니라 yield from을 사용해야 한다. 그리고 asyncio 코루틴은 yield from으로 호출하는 호출자에게 구동되거나, 이 장에서 설명할 asyncio.async() 등의 함수에 전달해서 구동해야 한다. 마지막으로 예제에서 보여주는 것처첨 @asyncio.coroutine 데커레이터를 코루틴에 적용해야 한다. 
```


In [9]:
""" [예제 18-2] spinner_asyncio.py : 코루틴으로 텍스트 스피너 애니메이트 하기 """

import asyncio
import itertools
import sys

@asyncio.coroutine # 사용할 코루틴을 데커레이트 처리함
def spin(msg):     # 예제 18-1의 singal 인수가 필요없음
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status)) 
        try:
            yield from asyncio.sleep(.1)
        except asyncio.CancelledError:
            break
    write(' ' * len(status) + '\x08' * len(status)) 

@asyncio.coroutine
def slow_function():
    yield from asyncio.sleep(3)
    return 42
    
@asyncio.coroutine
def supervision():
    # spin() 코루틴의 실행을 스케줄링하고 Task 객체 안에 넣어 Task 객체를 즉시 반환
    spinner = asyncio.async(spin('thinking!'))
    print('spinner object:', spinner)
    # slow_function () 함수를 구동해서 완료되면 반환된 값을 가져온다. 그러는 동안 이벤트 루프는 계속 실행됨
    result = yield from slow_function()
    spinner.cancel()
    return result

def main():
    # 이벤트 루프에 대한 참조를 가져옴
    loop = asyncio.get_event_loop()
    # supervisior() 코루틴을 구동해서 완료. 코루틴의 반환값은 run_util_complete() 메서드의 반환값이 됨
    result = loop.run_until_complete(supervision())
    # Task 객체는 cancel 메서드를 호출해서 취소 가능. 그러면 코루틴이 중단된 곳의 yield from 에서 asyncio.CancelledError 예외가 발행한다.
    loop.close()
    print('Answer', result)
    
# if __name__ == '__main__':
#     main()

<b> 예제 18-1, 18-2 supervisor( ) 함수의 차이 </b>
+ asyncio.Task는 threading.Thread와 거의 대등하다. 
+ Task는 코루틴을 구동하고, Thread는 콜러블을 호출한다.
+ Task 객체는 직접 생상하지 않고, 코루틴을 asyncio.asyn()나 loop.care_task()에 전달해서 가져온다.
+ Task 객체를 가져오면 이 객체는 이미 asyncio.async() 들에 의해 실행 스케줄링되어 있다. Thread 객체는 start() 메서드를 호출해서 힐행하라고 명령해야 한다.
+ 스레드화된 supervisior()에서 slow_function()은 평범한 함수로서 스레드에 의해 직접 호출된다. 비동기 supervisor()에서 slow_function()은 yield from으로 구동하는 코루틴이다.
+ 스레드는 외부에서 API를 이용해서 중단시킬 수 없다. 스레드르 아무 때나 중단시키면 시스템 상태의 무결성이 훼손되기 때문이다. Task에는 코루틴 안에 CancelledError를 발생시키는 Task.cancel() 객체 메서드가 있다. 코루틴은 중단되었던 yield 문에서 예외를 잡아서 처리할 수 있다.
+ supervisor() 코루틴은 main() 함수 안에서 loop.run_util_complete()로 실행해야 한다.

스레드는 스케줄러에 의해 언제드 스레드를 중단시킬 수 있으므로 크리티컬 섹션을 보고하기 위해 락을 잠그는 등 인터럽트되지 않게 해야하지만, 코루틴은 기본적으로 인터럽트로부터 보호된다. 제어권을 넘겨주고 싶을 때는 yield나 yield from을 이용해서 스케줄러에 넘겨줄 수 있다.

In [10]:
""" [예제 17-2] 순차 내려받기 스크립트, 몇몇 함수는 다른 스크립트에서 재사용할 것이다. """
import os
import time
import sys

import requests # 표준 라이브러리에 속해 있지 않으므로 관례에 따라 os, time, sys 표준 라이브러리 모듈을
                # 먼저 임포트 하고 한 줄 띄운 후 임포트한다.

POP20_CC = ('CN IN US ID BR PK NG BD RU JP '
            'MX PH VN ET EG DE IR TR CD FR').split()

BASE_URL = 'http://flupy.org/data/flags' # 국기 이미지를 갖고 있는 웹사이트
DEST_DIR = 'downloads/'                  # 이미지를 저장할 디렉토리

def save_flag(img, filename):
    """ 
    img(바이트 시퀀스)를 DEST_DIR 안의 filename으로 저장
    """
    path = os.path.join(DEST_DIR, filename)
    with open(path, 'wb') as fp:
        fp.write(img)

def show(text):
    """
    진행 상황을 한 줄로 출력한다.
    파이썬은 개행 문자를 받기 전까지 문자열을 출력하지 않으므로 sys.stdout.flush()를 호출해서 
    버퍼에 남아있는 내용을 모두 화면에 출력하게 한다.
    """
    print(text, end=' ')
    sys.stdout.flush()
    
def main(download_many):
    t0 = time.time()
    count = download_many(POP20_CC)
    elapsed = time.time() - t0
    msg = '\n{} flags downloaded in {:.2f}s'
    print(msg.format(count, elapsed))

In [11]:
import asyncio

import aiohttp # 별도 설치 필요

@asyncio.coroutine
def get_flag(cc):
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
    resp = yield from aiohttp.request('GET', url) # 블로킹 연산은 코루틴으로 구현되었고, yield from을 이용해서 이 코루틴에 위임하면, 코루틴이 비동기식으로 실행된다.
    image = yield from resp.read() # 응답 내용을 읽는 것은 별도의 비동기 연산에서 구현한다.
    return image

@asyncio.coroutine
def download_one(cc):
    image = yield from get_flag(cc) # 단어들을 yield from으로 가져온다는 것이 순차 버전의 download_one과 유일한 차이점
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc

def download_many(cc_list):
    loop = asyncio.get_event_loop() # 하위 이벤트 루프 구현에 대한 참조를 가져온다. 
    to_do = [download_one(cc) for cc in sorted(cc_list)] # 국기 이미지를 가져올 때마다 download_one() 함수를 호출해서 제너레이터 객체 리스트를 생성한다.
    wait_coro = asyncio.wait(to_do) # 일종의 코루틴으로 자신에게 전달된 코루틴들이 모두 완료되면 완료된다. 
    res, _ = loop.run_until_complete(wait_coro) # wait_coro가 완료될 때까지 이벤트 루프를 실행한다. 이 부분은 이벤트 루프가 실행하는 동안 블로킹된다. 
    loop.close() # 이벤트 루프를 종료한다.
    
    return len(res)

# if __name__ == '__main__':
#     main(download_many)

asyncio.wait() 코루틴은 Future 객체나 코루틴의 반복형을 받고, wait()은 각 코루틴을 Task 안에 래핑한다. 결국 wait()가 관리하는 모든 객체는 Future 객체가 된다. wait()는 코루틴 함수이기 때문에 이를 호출하면 코루틴/제너레이터 객체가 반환된다. wait_coro 변수에 들어 있는게 바로 wait()가 반환한 코루틴 제너레이터 객체이다. 이 코루틴을 구동하기 위해 loop.run_util_complete(run_util_complete()는 모두를 구동시킨다. )에 전달한다. 

loop.run_util_complete() 함수는 Future 객체나 코루틴을 받는다. 코루틴을 받으면 wait()과 비슷하게 코루틴을 Task 안에 래핑한다. 코루틴, Future, Task는 모두 yield from으로 구동할 수 있으며 run_util_complete()는 모두를 구동시킨다. wait_coro는 실행이 완료되면 (실행 완료 Future집합, 실행 미완료 Future집합) 튜플을 반환한다. 

참고로 wait()는 일부 Future 객체가 완료되지 않았더라도 반환하게 만드는 timeout과 return_when 키워드 전용 인수를 받는다. 자세한 사항은 asyncio.wait() 문서를 참조하라.

### 18.4 asyncio 내려받기 스크립드 개선
17.5절 '진행 상황을 출력하고 에러를 처리하며 내려받기'를 asyncio 버전에서도 적용한다.

#### 18.4.1 asyncio.as_completed( ) 사용하기
진행 상황을 보여주고 에러를 처리하기 위해 더 세세히 제어해야 하므로, 대부분의 논리를 donwload_many()에서 새로 만든 downloader_coro() 코루틴으로 옮기고 download_many()는 단지 이벤트 루프를 생성하고 downloader_coro() 스케줄링 하도록 수정한다.

In [12]:
""" [예제 18-7] 스크립트 앞부분 """

import asyncio
import collections 

import aiohttp
from aiohttp import web
import tqdm

DEFAULT_CONCUR_REQ = 5
MAX_CONCUR_REQ = 1000

class FetchError(Exception):
    """
    다른 HTTP와 네트워크 예외를 래핑하고 에러를 보고하기 위해 국가 코드를 보관하는 예외 클래스를 새로 정의한다.
    """
    def __init__(self, country_code):
        self.country_code = country_code
        
@asyncio.coroutine
def get_flag(base_url, cc):
    """
    내려받은 이미지 파일의 크기를 바이트 단위로 넘겨주거나, HTTP 응답 상태가 404일 때는 web.HTTPNotFound 예외를 생성하고,
    나머지 HTTP 에러코드에 대해서는 aiohttp.HttpProcessingError 예외를 발생시킨다.
    """
    url = '{}/{cc}/{cc}.gif'.format(base_url, cc=cc.lower())
    resp = yield from aiohttp.request('Get', url)
    if resp.status == 200:
        image = yield from resp.read()
        return image
    elif resp.status == 404:
        raise web.HTTPNotFound()
    else:
        raise aiohttp.HttpProcessingError(code=resp.status, message=resp.reason, headers=resp.headers)
    
@asyncio.coroutine
def download_one(cc, base_url, semaphore, verbose): # semaphore는 asyncio.Semaphore 객체이며, 동시 요청 수를 제한하기 위한 동기화 장치이다.
    try:
        with (yield from semaphore): #semaphore를 yield from 표현식의 콘텍스트 관리자로 사용하므로, 시스템 전체가 블로킹되지 않게 한다. 
            image = yield from get_flag(base_url, cc) # 단지 semaphore 카운터가 최대 허용 수에 이르렀을 때만 코루틴이 보장된다.
                                                      # with문을 나올 때 semaphore수 감소, 
                                                      # 이 세마포어 객체를 기다리는 다른 코루틴 객체가 진행되게 한다.
    except web.HTTPNotFound: # 파일을 못 찾은 상황
        status = HTTPStatus.not_found
        msg = 'not found'
    except Exception as exc: # 그 외의 이상상황
        raise FetchError(cc) from exc # 나머지 예외는 국가코드 및 raise X from Y 구문을 이용해서 원래 예외를 담은 FetchError로 만들어 전파한다.
    else:
        save_flag(image, cc.lower() + '.gif') # 이미지 국기를 디스크에 저장함
        status = HTTPStatus.ok
        msg = 'OK'
        
    if verbose and msg:
        print(cc, msg)
        
    return Result(status, cc)

In [13]:
@asyncio.coroutine
def downloader_coro(cc_list, base_url, verbose, concur_req): # 코루틴은 download_many()와 동일한 인수를 받지만 일반 함수가 아니라
                                                             # 코루틴 함수이므로 main()에서 바로 호출할 수 없다.
    counter = collections.Counter()
    semaphore = asyncio.Semaphore(concur_req) # 동시에 concor_req 개까지의 코루틴을 실행할 수 있게 해주는 세마포어를 생성한다.
    to_do = [download_one(cc, base_url, verbose, concur_req)
             for cc in sorted(cc_list)] # 코루틴을 호출하는 코루틴 객체의 리스트를 생성한다.

    to_do_iter = asyncio.as_completed(to_do) # 실행이 완료된 Future 객체를 반환하는 반복자를 가져온다.
    if not verbose:
        to_do_iter = tqdm.tqdm(to_do_iter, total=len(cc_list))
        
    for future in to_do_iter:
        try:
            res = yield from future # 객체의 결과를 가져올 때는 객체의 result() 메서드를 호출하는 것보다 yield from을 사용하는 것이 훨씬 더 쉽다.
        except FectchError as exc:  # download_one()에서 발생하는 모든 예외는 FetchError 객체안에 래핑되어 있다.
            counter_code = exc.country_code # 에러가 발생한 국가코드를 가져온다.
            try:
                error_msg = exc.__cause__.args[0] # 원래 예외에서 오류 메세지를 가져오려고 시도한다.
            except IndexError:
                error_msg = exc.__cause__.__class__.__name__ # 원래 예외에서 에러 메시지를 발견할 수 없으면 연결된 예외 클래스명을 에러 메시지로 사용한다.
            if verbose and error_msg:
                msg = '*** Error for {}: {}'
                print(msg.format(country_code, error_msg))
            status = HTTPStatus.error
        else:
            status = res.status
            
        counter[status] += 1
        
    return counter

def download_many(cc_list, bass_url, verbose, concur_req):
    loop = asyncio.get_event_loop()
    coro = downloader_coro(cc_list, base_url, verbose, concur_req)
    counts = loop.run_util_complete(coro) # 단순히 코루틴 객체를 생성하고 run_util_complete()를 이용해서 이벤트 루프에 전달한다.
    loop.close() # 모든 작업이 완료되면 이벤트 루프를 종료하고 카운터를 반환한다.
    
    return counts

# if __name__ == '__main__':
#     main(download_many, DEFAULT_CONCUR_REQ, MAX_CONCUR_REQ)

#### 18.4.2 Executor를 이용해서 이벤트 루프 블로킹 피하기
save_flag()는 asyncio 이벤트 루프와 공유하는 유일한 스레드를 블로킹하므로 파일을 저장하는 동안 애플리케이션 전체가 멈춘다. 이 문제의 해결책은 이벤트 루프 객체의 run_in_executor() 메서드이다.

In [14]:
""" [예제 18-9] 기본 스레드 풀 실행자를 이용해서 save_flag() 실행하기 """

@asyncio.coroutine
def download_one(cc, base_url, semaphore, verbose): # semaphore는 asyncio.Semaphore 객체이며, 동시 요청 수를 제한하기 위한 동기화 장치이다.
    try:
        with (yield from semaphore): #semaphore를 yield from 표현식의 콘텍스트 관리자로 사용하므로, 시스템 전체가 블로킹되지 않게 한다. 
            image = yield from get_flag(base_url, cc) # 단지 semaphore 카운터가 최대 허용 수에 이르렀을 때만 코루틴이 보장된다.
                                                      # with문을 나올 때 semaphore수 감소, 
                                                      # 이 세마포어 객체를 기다리는 다른 코루틴 객체가 진행되게 한다.
    except web.HTTPNotFound: # 파일을 못 찾은 상황
        status = HTTPStatus.not_found
        msg = 'not found'
    except Exception as exc: # 그 외의 이상상황
        raise FetchError(cc) from exc # 나머지 예외는 국가코드 및 raise X from Y 구문을 이용해서 원래 예외를 담은 FetchError로 만들어 전파한다.
    else:
        loop = asyncio.get_event_loop() # 이벤트 루프에 대한 참조자를 가져온다.
        loop.run_in_executor(None, # 첫번째 인수는 실행자 객체다. 이벤트 루프의 기본 스레드 풀 실행자를 사용할 때는 None으로 지정한다.
                             save_flag, image, cc.lower() + '.gif') # 나머지 인수는 콜러블 및 콜러블이 받을 위치 인수다.
        status = HTTPStatus.ok
        msg = 'OK'
        
    if verbose and msg:
        print(cc, msg)
        
    return Result(status, cc)

### 18.5 콜백에서 Future와 코루틴으로
#### 18.5.1 한 번 내려받을 때 여러 가지 요청하기
<b> download_one() </b>
+ yield from을 사용해서 get_flag() 및 새로 만들어진 get_country() 코루틴에 위임한다.

<b> get_flag() </b>
+ 기존 코드 대부분은 get_country()에서도 사용할 수 있도록 새로 만들어진 http_get() 코루틴으로 이동되었다.

<b> get_country() </b>
+ 이 코루틴은 국가 코드에 대한 metadata.json 파일을 가져와서 국가명을 꺼낸다.

<b> http_get() </b>
+ 웹에서 파일을 가져오기 위해 공통으로 사용하는 코드

In [15]:
""" [예제 18-13] 국기마다 두 개의 파일을 가져오기 위해 두 개의 코루틴에 위임한다. """

@asyncio.coroutine
def http_get(url):
    res = yield from asihttp.request('GET', url)
    if res.status == 200:
        ctype = res.header.get('Content-type', '').lower()
        if 'json' in ctype or url.endswith('json'): # 컨텐츠 자료형이 'json'이거나 url이 '.json'으로 끝나면
                                                    # 응답 객체의 json() 메서드를 호출해서 데이터를 파싱하고 파이썬 데이터 구조체를 반환한다.
            data = yield from res.json() 
        else:
            data = yield from res.read() # json 형식이 아니면 바이트 그대로 가져온다.
        return data
    
    elif res.status == 404:
        raise web.HTTPNotFound()
    else:
        raise aiohttp.errors.HttpProcessingError(code=res.status, message=res.reason, headers=res.headers)

@asyncio.coroutine
def get_country(base_url, cc):
    url = '{}/{cc}/metadata.json'.format(base_url, cc=cc.lower())
    metadata = yield from http_get(url)
    return metadata['country']

@asyncio.coroutine
def get_flag(base_url, cc):
    url = '{}/{cc}/{cc}.gif'.format(base_url, cc=cc.lower())
    return (yield from http_get(url))

@asyncio.coroutine
def downloads_one(cc, base_url, semaphore, verbose):
    try:
        with (yield from semaphore): # 전송속도 향상을 위해 semaphore에 의해 제어되는 두 개의 with블록에서 get_flag()와 get_country()를 따로 호출한다.
            image = yield from get_flag(base_url, cc)
        with (yield from semaphore):
            country = yield from get_country(base_url, cc)
    except web.HTTPNotFound:
        status = HTTPStatus.not_found
        msg = 'not found'
    except Exception as exc:
        raise FetchError(cc) from exc
    else:
        country = country.replace(' ', '_')
        filename = '{}-{}.gif'.format(country, cc)
        loop = asyncio.get_event_loop()
        loop.run_in_executor(None, save_flag, image, filename)
        msg = 'OK'

    if verbose and msg:
        print(cc, msg)

    return Result(status, cc)

### 18.6 asyncio 서버 작성
#### 18.6.1 asyncio TCP 서버


In [ ]:
""" [예제 18-14] asyncio.start_server()를 사용한 간단한 TCP 서버 """
import sys
import asyncio

from charfinder import UnicodeNameIndex # 문자영 인덱스를 생성하고 쿼리 메서드를 제공한다.

CRLF = b'\r\n'
PROMPT = b'?>'

index = UnicodeNameIndex() # 객체를 생성할 때 charfinder_index.pickle이 없는 경우에는 인덱스를 빌드하므로 시간이 걸릴 수 있다.

@asyncio.coroutine
def handle_gueries(reader, writer):
    while True: # 클라이언트에서 제어 문자를 받을 때까지 이 루프는 계속 세션을 처리한다.
        writer.write(PROMPT) # 코루틴이 아니라 일반함수이며 '?>' 문자열 프롬프트를 전송한다.
        yield from writer.drain() # 출력 버퍼를 플러시한다. 이 메서드는 코루틴이므로 yield from으로 호출해야 한다.
        data = yield from reader.readline() # 코루틴이며 bytes형을 반환한다.
        try:
            query = data.decode().strip()
        except UnicodeDecodeError: # 텔넷 클라이언트가 제어 문자를 보내면 UnicodeDecodeError 예외가 발생한다.
            query = '\x00'         # 간편함을 위해 제어 문자를 받으면 널 문자가 보내진 것처럼 처리한다. 
        client = writer.get_extra_info('peername')
        print('Received from {}: {!r}'.format(client, query))
        if query:
            if ord(query[:1]) < 32: # 제어 문자나 널 문자를 받으면 루프를 종료한다.
                
        
        
        
    